In [1]:
from tf_agents.drivers import py_driver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

#import environment
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment #allows parallel computing for generating experiences
from tf_agents.environments.parallel_py_environment import ParallelPyEnvironment

#import replay buffer
from tf_agents import replay_buffers as rb

#import agent
from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.utils import value_ops
from tf_agents.trajectories import StepType


#other used packages
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from matplotlib.axes import Axes as ax
import numpy as np
import tensorflow as tf
import tf_agents
from tf_agents.networks import actor_distribution_network
from tf_agents.specs import array_spec,tensor_spec
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.categorical_projection_network import CategoricalProjectionNetwork
#from custom_normal_projection_network import NormalProjectionNetwork
import os,gc


2025-07-07 12:28:42.027958: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 12:28:42.029961: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-07 12:28:42.065303: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-07 12:28:42.066439: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-07 12:28:42.661075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
#To limit TensorFlow to CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
cpus = tf.config.experimental.list_physical_devices('CPU') 
tf.config.experimental.set_visible_devices(cpus[0], 'CPU')
#enable multiprocessing for parallel computing
tf_agents.system.multiprocessing.enable_interactive_mode()
gc.collect()

2025-07-07 12:28:52.177692: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


0

#### Comment 1
In this way, set the reward function. it also the function for further test. 

In [3]:
###############
#Define the objective f to be maximized
N = 3   #This the dimension number
state_dim = N
def f(x,lambda_coef=0.3):
    return np.sum(-(x**2 -1)**2 - lambda_coef*(x-1)**2 + 5)

#### Comment 2
In this way, setting the starting value as x0_reinforce
sub_episode is the serial in a monte carlo model, so the mento carlo is running six time in each generation.
in total 
sub_episode_length * sub_episode_num steps

althoug the episode num need consider the parallel enviroment num

In [4]:
################
#Set initial x-values for all three algorithms
x0_reinforce = np.array([-1.5]*N,dtype=np.float32)
final_reward   = -1e9
final_solution = x0_reinforce.copy()    # so it always exists

sub_episode_length = 30 #number of time_steps in a sub-episode. 
episode_length = sub_episode_length*10  #an trajectory starts from the initial timestep and has no other initial timesteps
                                      #each trajectory will be split to multiple episodes
env_num = 6  #Number of parallel environments, each environment is used to generate an episode
print('x0', x0_reinforce)

x0 [-1.5 -1.5 -1.5]


In [5]:
generation_num = 100  #number of theta updates for REINFORCE-IP
disc_factor = 1.0
alpha = 0.2 #regularization coefficient
param_alpha = 0.15 #regularization coefficient for actor_network #tested value: 0.02
sub_episode_num = int(env_num*(episode_length/sub_episode_length)) #number of sub-episodes used for a single update of actor_net params
print("number of sub_episodes used for a single param update:", sub_episode_num)

number of sub_episodes used for a single param update: 60


In [6]:
class lr_schedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, C):
        self.initial_learning_rate = initial_lr
        self.C = C
    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        return self.initial_learning_rate*self.C/(self.C+step)
lr = lr_schedule(initial_lr=0.00001, C=100000)   
opt = tf.keras.optimizers.SGD(learning_rate=lr)

In [7]:
act_min = np.array([-1]*N, dtype=np.int32)
act_max = np.array([1]*N, dtype=np.int32)
#x0_reinforce = act_min

In [8]:
train_step_num = 0
step_size = np.array([0.1, 0.1, 0.1], dtype=np.float32)

In [9]:
def compute_reward(x):
    return f(x)

#### Comment 3
This part we have a key factor that distinguish to main project. use the MultiCategoricalProjectionNetwork
to support the action here, in the origin project, the CategoricalProjectionNetwork only able to support the action dimensions
that supports the same action space. This is not our case, we have tha action, require the different action space.
There are three method to solve this,<br/>
1, simply flat it to one array(flat_array.txt). https://github.com/tensorflow/agents/issues/702, but may account parameters explorion.<br/>
2, use muti-head MultiCategoricalProjectionNetwork. https://gist.github.com/sidney-tio/66abada949f1b629dd9ee28777d402d5<br/>
3, turing into the continous space<br/>

The method might become parameters exploration if action space go so high, therefore here, we try the method 2

In [10]:
from multicategorical_projection import MultiCategoricalProjectionNetwork
#from MultiCategoricalProjectionNetwork_T import MultiCategoricalProjectionNetwork_T

#### Comment 4
It worth dive into the real update process in the step function below.<br/>
The update is according to the equation: <br/>
self._state = self._state + (action-act_max)*step_size <br/>
then using the state calcualted the reward. <br/>
R = compute_reward(self._state) <br/>
The thought behind the behaviour is due to the directly use the action as state observation might create unstable duing to the spare samping, thus, instead replace the state with the change direction.
after readjust the action space, act_max will work as the middle value to judge it is the postive or negative step size in next step. 

In [11]:
class Env(py_environment.PyEnvironment):
    def __init__(self):
        '''The function to initialize an Env obj.
        '''
        #Specify the requirement for the value of action, (It is a 2d-array for this case)
        #which is an argument of _step(self, action) that is later defined.
        #tf_agents.specs.BoundedArraySpec is a class.
        #_action_spec.check_array( arr ) returns true if arr conforms to the specification stored in _action_spec
        #self._action_spec = array_spec.BoundedArraySpec(
        #                    shape=(state_dim,), dtype=np.int32, 
        #                    minimum=np.array([0]*N), 
        #                    maximum=act_max-act_min, 
        #                    name='action') #a_t is an 2darray

        self._action_spec = array_spec.BoundedArraySpec(
                                shape=(state_dim,),
                                dtype=np.int32,
                                minimum=np.array([0]*N),
                                maximum=act_max-act_min,
                                name='action')
    
        #Specify the format requirement for observation (It is a 2d-array for this case), 
        #i.e. the observable part of S_t, and it is stored in self._state
        self._observation_spec = array_spec.BoundedArraySpec(
                                 shape=(state_dim,), dtype=np.float32, name='observation') #default max and min is None
        self._state = np.array(x0_reinforce,dtype=np.float32)
        #self.A = mat
        self._episode_ended = False
        #stop_threshold is a condition for terminating the process for looking for the solution
        #self._stop_threshold = 0.01
        self._step_counter = 0

    def action_spec(self):
        #return the format requirement for action
        return self._action_spec

    def observation_spec(self):
        #return the format requirement for observation
        return self._observation_spec

    def _reset(self):
        self._state = np.array(x0_reinforce,dtype=np.float32)  #initial state
        self._episode_ended = False
        self._step_counter = 0
        
        #Reward
        initial_r = np.float32(0.0)
        
        #return ts.restart(observation=np.array(self._state, dtype=np.float32))
        return ts.TimeStep(step_type=StepType.FIRST, 
                           reward=initial_r, 
                           discount=np.float32(disc_factor), 
                           observation=np.array(self._state, dtype=np.float32)
                           )
    
    def set_state(self,new_state):
        self._state = new_state
    
    def get_state(self):
        return self._state
    
    def _step(self, action):
        '''
        The function for the transtion from (S_t, A_t) to (R_{t+1}, S_{t+1}).
    
        Input.
        --------
        self: contain S_t.
        action: A_t.
    
        Output.
        --------
        an TimeStep obj, TimeStep(step_type_{t+1}, R_{t+1}, discount_{t+1}, observation S_{t+1})
        ''' 
        # Suppose that we are at the beginning of time t 
        
        ################## --- Determine whether we should end the episode.
        if self._episode_ended:  # its time-t value is set at the end of t-1
            return self.reset()
        # Move on to the following if self._episode_ended=False
     
        
        ################# --- Compute S_{t+1} 
        #Note that we set the action space as a set of non-negative vectors
        #action-act_max converts the set to the desired set of negative vectors.

        self._state = self._state + (action-act_max)*step_size    
        self._step_counter +=1
        
        ################# --- Compute R_{t+1}=R(S_t,A_t)
        R = compute_reward(self._state)
        
        #Set conditions for termination
        if self._step_counter>=sub_episode_length-1:
            self._episode_ended = True  #value for t+1

        #Now we are at the end of time t, when self._episode_ended may have changed
        if self._episode_ended:
            #if self._step_counter>100:
            #    reward += np.float32(-100)
            #ts.termination(observation,reward,outer_dims=None): Returns a TimeStep obj with step_type set to StepType.LAST.
            return ts.termination(np.array(self._state, dtype=np.float32), reward=R)
        else:
            #ts.transition(observation,reward,discount,outer_dims=None): Returns 
            #a TimeStep obj with step_type set to StepType.MID.
            return ts.transition(np.array(self._state, dtype=np.float32), reward=R, discount=disc_factor)

##### for ENV test

##### test1 

In [ ]:
test_env = tf_py_environment.TFPyEnvironment(Env(), check_dims=True) #instance

In [ ]:
init_ts = test_env.reset()
x0 = init_ts.observation.numpy()[0]  

# construct all “unit” moves in each axis
# e.g. if your discrete bins are [-3, -2, …, +3], Δ=±1 in each dim
deltas = []
for d in range(state_dim):
    e = np.zeros_like(x0)
    e[d] = 1   # +1 step in dim d
    deltas.append(e)
    deltas.append(-e)  # also try −1

print("Flat-line test at x0 =", x0)
for Δ in deltas:
    r0 = compute_reward(x0)
    r1 = compute_reward(x0 + Δ)
    print(f"  Δ={Δ},  Δr = {r1 - r0:.6f}")

##### test2 

In [ ]:
def sample_random_action(spec):
    """Draw one uniform random integer action respecting a BoundedArraySpec."""
    # spec.minimum and spec.maximum may be scalars or arrays
    low = np.array(spec.minimum, dtype=spec.dtype)
    high = np.array(spec.maximum, dtype=spec.dtype)
    # +1 because np.randint upper bound is exclusive
    return np.random.randint(low, high + 1, size=spec.shape, dtype=spec.dtype)

def test_env():
    np.random.seed(0)
    env = Env()

    print("=== ACTION SPEC ===")
    print(env.action_spec())
    print("\n=== OBSERVATION SPEC ===")
    print(env.observation_spec())
    print()

    # 1) RESET 
    ts = env.reset()
    print("RESET ->")
    print("  state    =", ts.observation)
    print("  reward   =", ts.reward)
    print("  stepType =", ts.step_type)
    print()

    # 2) STEP RANDOMLY up to sub_episode_length+2
    max_steps = 10
    for i in range(max_steps):
        a = sample_random_action(env.action_spec())
        ts = env.step(a)
        print(f"STEP {i+1:2d}")
        print("  action   =", a)
        print("  new state=", ts.observation)
        print("  reward   =", ts.reward)
        print("  stepType =", ts.step_type)
        print()
        if ts.step_type == StepType.LAST:
            print(f"Episode terminated at step {i+1}.")
            break
    else:
        print(f"Did not terminate in {max_steps} steps.")

In [ ]:
np.random.seed(0)
test_env()

##### Env test END

#### Comment 5
train and eval happen in the same iteration, eval is like a examination on the current train result, in this way got the extra debugging process to look at the chaos come from.

In [12]:
parallel_env = ParallelPyEnvironment(env_constructors=[Env]*env_num, 
                                     start_serially=False,
                                     blocking=False,
                                     flatten=False
                                    )
#Use the wrapper to create a TFEnvironments obj. (so that parallel computation is enabled)
train_env = tf_py_environment.TFPyEnvironment(parallel_env, check_dims=True) #instance of parallel environments
eval_env = tf_py_environment.TFPyEnvironment(Env(), check_dims=False) #instance
# train_env.batch_size: The batch size expected for the actions and observations.  
print('train_env.batch_size = parallel environment number = ', env_num)

###Without the Parallel
#train_env = tf_py_environment.TFPyEnvironment(Env(), check_dims=True) #instance of parallel environments
#eval_env = tf_py_environment.TFPyEnvironment(Env(), check_dims=False) #instance

train_env.batch_size = parallel environment number =  6


#### comment 6 How to set network
On the how to set the neural network in setting.
The neural network is create the following structure use the code for actor_net

Input → Dense(10) → Activation(tanh) → Dense(5) → Activation(tanh) → [projection head]

Input is the observation_spec, then the projection head is the action_spec. Unlike the supervised learning process, it trains with the forward and back prop with the loop. The reinforcement actually like a delay update, decouple the data collection(mento-calo process) and gradient update(using loss function to gradient update), the reward of vanilla reinforcement(pure Monte Carlo without bootstraping or n-step return these kind of policy optimal algorithm) only sum up in the end of the episode.

in the discrete_projection_net, persumming we put a muti-dimesion predict layer here to predict the action, the place it put is after the Dense(5) layer above it send to this special neural layer.

--Dense(5)-->   (batch,  5)  --tanh--> CategoricalNetwork -> projection head

#### comment 7 CategoricalProjectionNetwork and MultiCategoricalProjectionNetwork
The general idea of this Categorical network is to build the layer, that mapping to the all action point to the logits(the neural on the network to predicts the value), which suppose to be wired up in the policy network. The higher policy should able to do two seperate output, a log probability and a action sampler. 

The init() and call() function all invoke by the higher policy netwrok, like ActorDistributionNetwork here it's leave the set on the final layer it applied "discrete_projection_net=" before the projection head. actually the class CategoricalProjectionNetwork(network.DistributionNetwork), it use the inheritance here, then later it will reinvoke the paranet method by super(CategoricalProjectionNetwork, self).__init__. The actual init() should happened when intitial do the Network build ActorDistributionNetwork(...), although the call() function is invoked when the policy is on working scenario. The call() is warped up in the driver and the agent, in the <br/>
action_step = collect_policy.action(time_step, policy_state)

The main different is on the multi-head one, because the dimension are different, they need extra split on the netwrok. The detail can see in the _output_distribution_spec in the multi-head situation, it runs with the further MultiCategoricalDistributionBlock, use to spilt more sub-logits, according to the structure. Also there is the extra components in call function, it related to the mask(some special scenario to create the condtions), I believe this project do not have this feature here.

extra: The observation and rewards, also with the different, consider the example in original reward, def f(x,lambda_coef=0.3):return -(x**2 -1)**2 - lambda_coef*(x-1)**2 + 5, actually able to return the as many dimension of the reward as the input action, in contrast, the scenario we want use the netwrok to reture a single reward value. like with np.sum()

In [13]:
#actor_distribution_network outputs a distribution
#it is a neural net which outputs the parameter (mean and sd, named as loc and scale) for a normal distribution
tf.random.set_seed(0)
actor_net = actor_distribution_network.ActorDistributionNetwork(   
                                         train_env.observation_spec(),
                                         train_env.action_spec(),
                                         fc_layer_params=(10,5), #Hidden layers
                                         seed=0, #seed used for Keras kernal initializers for NormalProjectionNetwork.
                                         discrete_projection_net=MultiCategoricalProjectionNetwork,
                                         activation_fn = tf.math.tanh,
                                         #continuous_projection_net=(NormalProjectionNetwork)
                                         )


In [14]:
#Create the  REINFORCE_agent
train_step_counter = tf.Variable(0)
tf.random.set_seed(0)

# REINFORCE_agent = reinforce_agent.ReinforceAgent(
#        time_step_spec = train_env.time_step_spec(),
#        action_spec = train_env.action_spec(),
#        actor_network = actor_net,
#        value_network = None,
#        value_estimation_loss_coef = 0.2,
#        optimizer = opt,
#        advantage_fn = None,
#        use_advantage_loss = False,
#        gamma = 1.0, #discount factor for future returns
#        normalize_returns = True, #The instruction says it's better to normalize
#        gradient_clipping = 1.0,
#        entropy_regularization = 0.05,
#        train_step_counter = train_step_counter
#        )


#Original agent
REINFORCE_agent = reinforce_agent.ReinforceAgent(
       time_step_spec = train_env.time_step_spec(),
       action_spec = train_env.action_spec(),
       actor_network = actor_net,
       value_network = None,
       value_estimation_loss_coef = 0.2,
       optimizer = opt,
       advantage_fn = None,
       use_advantage_loss = False,
       gamma = 1.0, #discount factor for future returns
       normalize_returns = False, #The instruction says it's better to normalize
       gradient_clipping = None,
       entropy_regularization = 0.01,
       train_step_counter = train_step_counter
       )

REINFORCE_agent.initialize()

##### Examine the actor net

In [ ]:
# High-level Keras summary
actor_net.summary()

# Dig into the projection sub-network
print("\n=== projection sub-network ===")
proj_net = actor_net._projection_networks
print(proj_net)
# If it’s a Keras Model, you can do:
try:
    proj_net.summary()
except Exception:
    # not all TF-Agents networks expose .summary()
    print("No .summary() on this object; listing its layers instead:")
    for layer in proj_net.layers:
        print(" ", layer.name, layer)

# Finally, list all trainable variables so you see exactly which weight matrices live where
print("\n=== trainable variables ===")
for v in actor_net.trainable_variables:
    print(v.name, v.shape)


##### end of examination

#### Comment 7 The max_length has optimised
Left the episode_length*env_num*2 rather than episode_length*100 should large enough for the headroom.
reply buffer use in the trainning procedure, and clean it self in each epoch. 

##### Agent Policy Test

In [ ]:
##2) Sampling vs. greedy policy check
##Verify you really are using a stochastic policy for collection, and that it actually samples other bins early on:
##If all 10 samples equal greedy (or all equal your initial bin), then you’re accidentally collecting with the deterministic policy or your logits already collapsed.

ts = train_env.reset()
print("Greedy vs. sampled at initial state:")
# greedy “best‐guess” action
greedy = REINFORCE_agent.policy.action(ts).action.numpy()
print("  greedy action =", greedy)

# 10 stochastic samples from collect_policy
for i in range(10):
    a = REINFORCE_agent.collect_policy.action(ts).action.numpy()
    print(f"  sample {i:2d} →", a)

##### End Policy Test

In [15]:
#################
#replay_buffer is used to store policy exploration data
#################
replay_buffer = rb.TFUniformReplayBuffer(
                data_spec = REINFORCE_agent.collect_data_spec,  # describe spec for a single iterm in the buffer. A TensorSpec or a list/tuple/nest of TensorSpecs describing a single item that can be stored in this buffer.
                batch_size = env_num,    # number of parallel worlds, where in each world there is an agent generating trajectories
                                         # One batch corresponds to one parallel environment
                
                max_length = 100    
    
                # The maximum number of items that can be stored in a single batch segment of the buffer.     
                                                    # if exceeding this number previous trajectories will be dropped
)

#test_buffer is used for evaluating a policy
test_buffer = rb.TFUniformReplayBuffer(
                data_spec= REINFORCE_agent.collect_data_spec,  # describe a single iterm in the buffer. A TensorSpec or a list/tuple/nest of TensorSpecs describing a single item that can be stored in this buffer.
                batch_size= 1,    # number of parallel worlds, where in each world there is an agent generating trajectories
                                                    # train_env.batch_size: The batch size expected for the actions and observations.  
                                                    # batch_size: Batch dimension of tensors when adding to buffer. 
                max_length = episode_length         # The maximum number of items that can be stored in a single batch segment of the buffer.     
                                                    # if exceeding this number previous trajectories will be dropped
)


#### Comment 8
In which way to select a proper policy. The chosen on the driver is not the main concern. It is mainly how you want to trainning in the which steps level. DynamicEpisodeDriver is more suitabale for trainning in the parallel env, pydriver is sutiable to the single episode, there are other drivers existing such as,  DynamicStepDriver(able to control the exactly how many steps want to run), onstepdriver(only run one steps). 

really distinguish the trainning and evaluation is on how to set the policy dependence.<br/>
policy = REINFORCE_agent.collect_policy, stochastic policy<br/>
policy = REINFORCE_agent.policy,greedy policy<br/>

generally, the trainning is on the stochastic policy collection, but the evaluation is on the greedy policy picks arg⁡max⁡𝑎 𝜋𝜃(𝑎∣𝑠)



In [16]:
#A driver uses an agent to perform its policy in the environment.
#The trajectory is saved in replay_buffer
collect_driver = DynamicEpisodeDriver(
                                     env = train_env, #train_env contains parallel environments (no.: env_num)
                                     policy = REINFORCE_agent.collect_policy,
                                     observers = [replay_buffer.add_batch],
                                     num_episodes = sub_episode_num   #SUM_i (number of episodes to be performed in the ith parallel environment)
                                    )

#For policy evaluation
test_driver = py_driver.PyDriver(
                                     env = eval_env, #PyEnvironment or TFEnvironment class
                                     policy = REINFORCE_agent.policy,
                                     observers = [test_buffer.add_batch],
                                     max_episodes=1, #optional. If provided, the data generation ends whenever
                                                      #either max_steps or max_episodes is reached.
                                     max_steps=sub_episode_length
                                )

In [17]:
#Functions needed for training
def extract_episode(traj_batch,epi_length,attr_name = 'observation'):
    """
    This function extract episodes (each episode consists of consecutive time_steps) from a batch of trajectories.
    Inputs.
    -----------
    traj_batch:replay_buffer.gather_all(), a batch of trajectories
    epi_length:int, number of time_steps in each extracted episode
    attr_name:str, specify which data from traj_batch to extract
    
    Outputs.
    -----------
    tf.constant(new_attr,dtype=attr.dtype), shape = [new_batch_size, epi_length, state_dim]
                                         or shape = [new_batch_size, epi_length]
    """
    attr = getattr(traj_batch,attr_name)
    original_batch_dim = attr.shape[0]
    traj_length = attr.shape[1]
    epi_num = int(traj_length/epi_length) #number of episodes out of each trajectory
    batch_dim = int(original_batch_dim*epi_num) #new batch_dim
    
    if len(attr.shape)==3:
        stat_dim = attr.shape[2]
        new_attr = np.zeros([batch_dim, epi_length, state_dim])
    else:
        new_attr = np.zeros([batch_dim, epi_length])
        
    for i in range(original_batch_dim):
        for j in range(epi_num):
            new_attr[i*epi_num+j] = attr[i,j*epi_length:(j+1)*epi_length].numpy()
        
    return tf.constant(new_attr,dtype=attr.dtype)

# Please also see the metrics module for standard implementations of different
# metrics.

##### Test Trainning

In [ ]:
#%%timeit
##### - Train REINFORCE_agent's actor_network multiple times.
update_num = 3
eval_intv = 1 #number of updates required before each policy evaluation
REINFORCE_logs = [] #for logging the best objective value of the best solution among all the solutions used for one update of theta
final_reward = -1000
plot_intv = 500

tf.random.set_seed(0)
for n in range(0,update_num):
    #Generate Trajectories
    replay_buffer.clear()
    collect_driver.run()  #a batch of trajectories will be saved in replay_buffer
    
    experience = replay_buffer.gather_all() #get the batch of trajectories, shape=(batch_size, episode_length)
    rewards = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'reward') #shape=(sub_episode_num, sub_episode_length)
    observations = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'observation') #shape=(sub_episode_num, sub_episode_length, state_dim)
    actions = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'action') #shape=(sub_episode_num, sub_episode_length, state_dim)
    step_types = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'step_type')
    discounts = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'discount')
    
    time_steps = ts.TimeStep(step_types,
                             tf.zeros_like(rewards),
                             tf.zeros_like(discounts),
                             observations
                            )
    
    rewards_sum = tf.reduce_sum(rewards, axis=1) #shape=(sub_episode_num,)
    
    with tf.GradientTape() as tape:
        #trainable parameters in the actor_network in REINFORCE_agent
        variables_to_train = REINFORCE_agent._actor_network.trainable_weights
    
        ###########Compute J_loss = -J
        actions_distribution = REINFORCE_agent.collect_policy.distribution(
                               time_steps, policy_state=None).action


        ########################Check the action distribution
        #######
        logits = actions_distribution.parameters["logits"]
        
        print("min / max / mean logits =",
              tf.reduce_min(logits).numpy(),
              tf.reduce_max(logits).numpy(),
              tf.reduce_mean(logits).numpy())
        #######
        ########################
        
    
        #log(pi(action|state)), shape = (batch_size, epsode_length)
        action_log_prob = common.log_probability(actions_distribution, 
                                                 actions,
                                                 REINFORCE_agent.action_spec)
    
        J = tf.reduce_sum(tf.reduce_sum(action_log_prob,axis=1)*rewards_sum)/sub_episode_num
        
        ###########Compute regularization loss from actor_net params
        regu_term = tf.reduce_sum(variables_to_train[0]**2)
        num = len(variables_to_train) #number of vectors in variables_to_train
        for i in range(1,num):
            regu_term += tf.reduce_sum(variables_to_train[i]**2)
        
        total = -J + param_alpha*regu_term
    
    #update parameters in the actor_network in the policy
    grads = tape.gradient(total, variables_to_train)
    grads_and_vars = list(zip(grads, variables_to_train))
    opt.apply_gradients(grads_and_vars=grads_and_vars)
    train_step_num += 1

    batch_rewards = rewards.numpy()
    batch_rewards[:,-1] = -np.power(10,8) #The initial reward is set as 0, we set it as this value to not affect the best_obs_index 
    best_step_reward = np.max(batch_rewards)
    best_step_index = [int(batch_rewards.argmax()/sub_episode_length),batch_rewards.argmax()%sub_episode_length+1]
    best_step = observations[best_step_index[0],best_step_index[1],:] #best solution
    #best_step_reward = f(best_solution)
    avg_step_reward = np.mean(batch_rewards[:,0:-1])
    REINFORCE_logs.append(best_step_reward)
    
    if final_reward is None or best_step_reward>final_reward:
        print("final reward before udpate:",final_reward)
        final_reward = best_step_reward
        final_solution = best_step.numpy()
        print("final reward after udpate:",final_reward)
        print('updated final_solution=', final_solution)
    
    #print(compute_reward(best_obs,alpha))
    if n%eval_intv==0:
        print("train_step no.=",train_step_num)
        print('best_solution of this generation=', best_step.numpy())
        print('best step reward=',best_step_reward.round(3),f(best_step.numpy()))
        print('avg step reward=', round(avg_step_reward,3))
        #print('act_logits:', actions_distribution._logits.numpy()[0] ) #second action mean
        print('best_step_index:',best_step_index)
        #print('collect_traj:', observations[0,:,0].numpy())
        ##  I persume this means the a random trajectories give in the scenario
        print('collect_traj:', observations[0,:].numpy())
        
        #Test trajectory generated by mode of action_distribution in agent policy
        test_buffer.clear()
        test_driver.run(eval_env.reset())  #generate batches of trajectories with agent.collect_policy, and save them in replay_buffer
        experience = test_buffer.gather_all()  #get all the stored items in replay_buffer
        test_trajectory = experience.observation.numpy()[0,:]
        print('test_traj', test_trajectory)
        
               
print('final_solution=',final_solution,
      'final_reward=',final_reward,
      )
REINFORCE_logs = [max(REINFORCE_logs[0:i]) for i in range(1,generation_num+1)] #rolling max

In [ ]:
observations[0,:].shape

In [ ]:
batch_observations = observations.numpy()

In [ ]:
batch_rewards[:,0]

In [ ]:
observations[1,:]

In [ ]:
batch_observations[:,3]

In [ ]:
f(np.array([-2.6,-2.3,-2.7]))

In [ ]:
np.array([1, 1, 1]-act_max)*step_size

##### end of test

In [20]:
##### - Train REINFORCE_agent's actor_network multiple times.
update_num = 500 
eval_intv = 30 #number of updates required before each policy evaluation
REINFORCE_logs = [] #for logging the best objective value of the best solution among all the solutions used for one update of theta
#final_reward = -1000
plot_intv = 500

tf.random.set_seed(0)
for n in range(0,update_num):
    #Generate Trajectories
    replay_buffer.clear()
    collect_driver.run()  #a batch of trajectories will be saved in replay_buffer
    
    experience = replay_buffer.gather_all() #get the batch of trajectories, shape=(batch_size, episode_length)
    rewards = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'reward') #shape=(sub_episode_num, sub_episode_length)
    observations = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'observation') #shape=(sub_episode_num, sub_episode_length, state_dim)
    actions = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'action') #shape=(sub_episode_num, sub_episode_length, state_dim)
    step_types = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'step_type')
    discounts = extract_episode(traj_batch=experience,epi_length=sub_episode_length,attr_name = 'discount')
    
    time_steps = ts.TimeStep(step_types,
                             tf.zeros_like(rewards),
                             tf.zeros_like(discounts),
                             observations
                            )
    
    rewards_sum = tf.reduce_sum(rewards, axis=1) #shape=(sub_episode_num,)

    
    with tf.GradientTape() as tape:
        #trainable parameters in the actor_network in REINFORCE_agent
        variables_to_train = REINFORCE_agent._actor_network.trainable_weights
    
        ###########Compute J_loss = -J
        actions_distribution = REINFORCE_agent.collect_policy.distribution(
                               time_steps, policy_state=None).action

        ######Entropy trace during “burn-in”, Watch the policy’s entropy during the very first few updates

        ent = actions_distribution.entropy().numpy()   # shape = (batch_size, state_dim)
        print(f"[step {n:3d}] mean entropy per-dim = {ent.mean():.4f}")
        #######
        #log(pi(action|state)), shape = (batch_size, epsode_length)
        action_log_prob = common.log_probability(actions_distribution, 
                                                 actions,
                                                 REINFORCE_agent.action_spec)
    
        J = tf.reduce_sum(tf.reduce_sum(action_log_prob,axis=1)*rewards_sum)/sub_episode_num
        
        ###########Compute regularization loss from actor_net params
        regu_term = tf.reduce_sum(variables_to_train[0]**2)
        num = len(variables_to_train) #number of vectors in variables_to_train
        for i in range(1,num):
            regu_term += tf.reduce_sum(variables_to_train[i]**2)
        
        total = -J + param_alpha*regu_term
    
    #update parameters in the actor_network in the policy
    grads = tape.gradient(total, variables_to_train)
    grads_and_vars = list(zip(grads, variables_to_train))
    opt.apply_gradients(grads_and_vars=grads_and_vars)
    train_step_num += 1
    
    batch_rewards = rewards.numpy()
    batch_rewards[:,-1] = -np.power(10,8) #The initial reward is set as 0, we set it as this value to not affect the best_obs_index 
    best_step_reward = np.max(batch_rewards)
    best_step_index = [int(batch_rewards.argmax()/sub_episode_length),batch_rewards.argmax()%sub_episode_length+1]
    best_step = observations[best_step_index[0],best_step_index[1],:] #best solution
    #best_step_reward = f(best_solution)
    avg_step_reward = np.mean(batch_rewards[:,0:-1])
    REINFORCE_logs.append(best_step_reward)
    
    if final_reward is None or best_step_reward>final_reward:
        print("final reward before udpate:",final_reward)
        final_reward = best_step_reward
        final_solution = best_step.numpy()
        print("final reward after udpate:",final_reward)
        print('updated final_solution=', final_solution)
    
    #print(compute_reward(best_obs,alpha))
    if n%eval_intv==0:
        print("train_step no.=",train_step_num)
        print('best_solution of this generation=', best_step.numpy())
        print('best step reward=',best_step_reward.round(3),f(best_step.numpy()))
        print('avg step reward=', round(avg_step_reward,3))
        #print('act_logits:', actions_distribution._logits.numpy()[0] ) #second action mean
        print('best_step_index:',best_step_index)
        print('collect_traj:', observations[0,:].numpy())
        
        #Test trajectory generated by mode of action_distribution in agent policy
        test_buffer.clear()
        test_driver.run(eval_env.reset())  #generate batches of trajectories with agent.collect_policy, and save them in replay_buffer
        experience = test_buffer.gather_all()  #get all the stored items in replay_buffer
        test_trajectory = experience.observation.numpy()[0,:]
        print('test_traj', test_trajectory)
        
               
print('final_solution=',final_solution,
      'final_reward=',final_reward,
      )
REINFORCE_logs = [max(REINFORCE_logs[0:i]) for i in range(1,generation_num+1)] #rolling max

[step   0] mean entropy per-dim = 3.1601
train_step no.= 1001
best_solution of this generation= [-0.8 -0.8 -0.9]
best step reward= 11.678 11.6777
avg step reward= 8.91
best_step_index: [16, 6]
collect_traj: [[-1.2        -1.1        -0.59999996]
 [-1.1        -1.2        -0.7       ]
 [-1.1        -1.1        -0.59999996]
 [-1.         -1.1        -0.5       ]
 [-1.         -1.         -0.39999998]
 [-0.9        -1.         -0.5       ]
 [-1.         -0.9        -0.39999998]
 [-0.9        -0.9        -0.29999998]
 [-1.         -0.9        -0.19999999]
 [-1.         -0.9        -0.29999998]
 [-1.5        -1.5        -1.5       ]
 [-1.6        -1.4        -1.4       ]
 [-1.5        -1.3        -1.4       ]
 [-1.4        -1.2        -1.4       ]
 [-1.3        -1.3        -1.4       ]
 [-1.2        -1.4        -1.3       ]
 [-1.1        -1.4        -1.2       ]
 [-1.         -1.3        -1.3       ]
 [-1.         -1.2        -1.3       ]
 [-0.9        -1.2        -1.2       ]
 [-0.8       

In [ ]:
f(np.array([-1.5,-1.5,-1.5]))

In [ ]:
batch_rewards[:,0:-1].shape

In [ ]:
batch_rewards[:,30]

In [ ]:
a = [[1,2,10],[3,4,13],[1,4,7],[1,9,102]]
a = np.array(a)
a

In [ ]:
a[:,0:1]
